In [1]:
import sys
import json
import os

sys.path.append(os.path.abspath("../src"))
PROJECT_ROOT = os.getenv("PROJECT_ROOT")

In [2]:
from recommender.data_processors.summoner_champion_processor import SummonerMixProcessor
from recommender.data_loaders.summoner_match_loader import SummonerMatchLoader
from recommender.utils.riot_api_helper import RiotApiHelper
from recommender.utils.map_helper import MapHelper

smp = SummonerMixProcessor()
rah = RiotApiHelper()
sml = SummonerMatchLoader()
mh = MapHelper()

In [3]:
puuid = "_2dRYSt03dHrJrRDcYImk2_umcnVSo8i-1WMun-j8YAFYk9CzauMu7WjQ8IJCY_RDWMopccQI_qfcw"
match_ids = rah.get_player_matches(puuid)

In [6]:
s = await(sml.async_dump_data_for_puuid(puuid))

Loading data for puuid: _2dRYSt03dHrJrRDcYImk2_umcnVSo8i-1WMun-j8YAFYk9CzauMu7WjQ8IJCY_RDWMopccQI_qfcw


In [4]:
# WAAAAY faster to load matches
import asyncio
import aiohttp

semaphore = asyncio.Semaphore(5)

async with aiohttp.ClientSession() as session:
    tasks = [
        rah.async_get_match_info(session, match_id, semaphore)
        for match_id in match_ids
    ]
    matches2 = await asyncio.gather(*tasks)

In [7]:
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
with ProcessPoolExecutor() as executor:
    # Partial function to fix puuid and processed_puuids args (optional)
    futures = [
        executor.submit(sml.process_match_for_puuid, match, puuid)
        for match in matches2
    ]

In [14]:
from collections import defaultdict
total_dict = defaultdict(int)
dicts = [f.result() for f in futures]

In [16]:
futures[0].result()

<coroutine object SummonerMatchLoader.process_match_for_puuid at 0x0000021A4E496740>

In [ ]:
# Step 1: Build the df with column of lets say mode rank, mode lanes, roles, attack_types
# Step 2: Make train and test loader using TwoTowerChampionDataset
# Step 3: iterate for user_feattures, champ_features, rating in loader: preds = model(user_features, champ_features)